This is an analysis over results consistent with Petrov et al. (2022)'s projections for 4 years of observing at A+ sensitivity, with updated sky location handling and a three-detector network (HLV). This may be the final version.

In [78]:
## point to my utils/analysis code
hbpmloc = '/mnt/c/Users/Alexander/Documents/LIGO/PM/hbpm_v2/'
## basic imports
import numpy as np
import matplotlib.pyplot as plt
import mplcyberpunk
import scipy.stats as st
import matplotlib
from scipy.interpolate import interp1d
from scipy.stats import gaussian_kde as kde
from scipy.stats.mstats import mquantiles as mq
from pesummary.core.plots.bounded_1d_kde import bounded_1d_kde
from glob import glob
import pandas as pd
import sys 
import os
import pickle
from astropy import cosmology as co
from astropy.units import *
## Importing the accompanying utility and analysis files
sys.path.append(os.path.abspath(hbpmloc+'hbpm_utils/'))
from hbpm_utils import *
from importlib import reload
reload(sys.modules["hbpm_utils"])
from hbpm_utils import *
sys.path.append(hbpmloc)
from hbpm_analysis import run_analysis
reload(sys.modules["hbpm_analysis"])
from hbpm_analysis import run_analysis

## my preferred figure settings
matplotlib.rcParams['figure.figsize'] = (8.08, 5.)
matplotlib.rcParams['xtick.labelsize'] = 12.0
matplotlib.rcParams['ytick.labelsize'] = 12.0
matplotlib.rcParams['axes.labelsize'] = 14.0
matplotlib.rcParams['legend.fontsize'] = 12
matplotlib.rcParams['axes.titlesize'] = 16
matplotlib.rcParams['axes.prop_cycle'] = matplotlib.cycler(color=["mediumorchid", "teal", "goldenrod","slategray"])

In [79]:
post_bandwidth = 0.15
prior_bandwidth = 0.4

In [80]:
## load the not-bootstrap bootstrap
## really need a new name for this
## should also make a function to do this & put the samples in the hbpm_v2 directory
coeffs = pd.read_csv(
    '/mnt/c/Users/Alexander/Documents/LIGO/PM/hbpm/analysis_2022/empirical_relation_samples/sampled_coefficients_v05.tab',
    sep=' ')
bootstrap = coeffs.to_numpy()

## Preliminaries

In [81]:
## fine grid
fs = np.linspace(1.5,4,200)
Ms = np.linspace(0.8,1.8,200)

In [82]:
## get the Dietrich et al. (2020) multimessenger prior for R_1.6
Rs_mm, Rprior_kernel_mm, R16samples_mm = load_Rprior(hbpmloc+'/priors/R16_prior.txt',return_samples=True,plot=False)

## uniform prior, consider R on [9,15]
Rs = np.linspace(9,15,200)

In [83]:
uniform_Rprior = st.uniform(loc=Rs.min(),scale=(Rs.max()-Rs.min()))

In [84]:
# ## get the updated built-in BW fpeak prior
# fprior_path = hbpmloc+'priors/fpeak_broad_O5_prior_100x.txt'
# fprior_samples = np.loadtxt(fprior_path)
# fprior = Prior_f(fprior_samples,boundary='Reflection',kde_bandwidth=prior_bandwidth)

In [85]:
# ## plot the fpeak prior
# plt.figure()
# plt.plot(fs,fprior.pdf(fs),label='$\\mathrm{f_{peak}}$ Prior KDE')
# plt.hist(fprior.samples,density=True,alpha=0.7,label='$\mathrm{f_{peak}}$'+' Prior Samples',bins=20)
# plt.plot(fs,st.uniform.pdf(fs,loc=1.5,scale=2.5),label='Uniform Distribution')
# plt.xlabel('$\mathrm{f_{peak}}$ (kHz)')
# plt.xlim(1.5,4)
# # plt.yticks([])
# plt.ylim(0.35,0.45)
# plt.title('Peak Frequency Prior')
# plt.legend(loc='lower right')
# # plt.savefig('fpeak_prior_forcapstone.jpg',bbox_inches='tight')
# plt.show()

## Analyses

In [86]:
priorfile = hbpmloc+'/priors/fpeak_O5_prior_bw0p4_downsampled.pickle'
savebase = '3-Aplus-4yr-HLV-072622'

### Part 1

In [ ]:
pt1_datadir = '/mnt/c/Users/Alexander/Documents/LIGO/PM/BWruns/HLV_runs/Aplus_4yr/part1/'
Rs_pt1,likes_pt1,post_pt1,stats_pt1,postdict_pt1 = run_analysis(pt1_datadir,priorfile,
                                                                'uniform','sfhx',
                                                                hbpmloc+'/observing_run_sims/Aplus_4yr_events_pt1.csv',
                                                               hbpmloc+'/nr_files/sfhx_event_parameters.csv',
                                                               saveto='./'+savebase+'_pt1/',bootstrap=bootstrap,
                                                                 prior_bandwidth=prior_bandwidth,
                                                                 posterior_bandwidth=post_bandwidth,
                                                                z_adj='known',Mchirp_scaling='snr',aggregation='sum')

Running a hierarchical Bayesian post-merger analysis...
Loading priors...
Loading simulation data...
Generating event dictionary...


 19%|=>>>>>>>>>| 34/178 [01:15<05:18,  2.22s/it]

### Part 2

In [ ]:
pt2_datadir = '/mnt/c/Users/Alexander/Documents/LIGO/PM/BWruns/HLV_runs/Aplus_4yr/part2/'
Rs_pt2,likes_pt2,post_pt2,stats_pt2,postdict_pt2 = run_analysis(pt2_datadir,
                                                                     priorfile,
                                                                'uniform','sfhx',
                                                                hbpmloc+'/observing_run_sims/Aplus_4yr_events_pt2.csv',
                                                               hbpmloc+'/nr_files/sfhx_event_parameters.csv',
                                                               saveto='./'+savebase+'_pt2/',bootstrap=bootstrap,
                                                                 prior_bandwidth=prior_bandwidth,
                                                                 posterior_bandwidth=post_bandwidth,
                                                                z_adj='known',Mchirp_scaling='snr',aggregation='sum')

## Combined

In [ ]:
Aplus_4yr_likes = [*likes_pt1,*likes_pt2]

In [ ]:
plt.figure()
ax = plt.gca()
# plot_aggregate_posterior_on_ax(Rs,likes_sly4,uniform_Rprior,Rtrue=11.54,ax=ax1,legend_loc='upper left',
#                          title='O4+O5 Posterior for sly4 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
plot_aggregate_posterior_on_ax(Rs,Aplus_4yr_likes,uniform_Rprior,Rtrue=11.98,ax=ax,legend_loc='upper left',
                         title='A+ 4yr Posterior for SFHX EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# plot_aggregate_posterior_on_ax(Rs,likes_dd2,uniform_Rprior,Rtrue=13.26,ax=ax3,legend_loc='upper left',
#                          title='O4+O5 Posterior for dd2 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# ax1.set_ylim()
# plt.tight_layout()
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.png',bbox_inches='tight')
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.pdf',bbox_inches='tight')
plt.show()

In [ ]:
pt1_postdict = load_posterior_pickle('./'+savebase+'_pt1/posterior_eventdict.pickle')

In [ ]:
badkeys = ['eos', 'Rs', 'prior']

In [ ]:
goodkeys_pt1 = [key for key in pt1_postdict.keys() if key not in badkeys]

In [ ]:
unnormed_likes_pt1 = []
for key in goodkeys_pt1:
    unnormed_likes_pt1.append(pt1_postdict[key]['likelihood_i'])

In [ ]:
plot_aggregate_posterior(Rs,unnormed_likes_pt1,uniform_Rprior,Rtrue=11.98)

In [ ]:
# help(load_posterior_pickle)
pt2_postdict = load_posterior_pickle('./'+savebase+'_pt2/posterior_eventdict.pickle')

In [ ]:
goodkeys_pt2 = [key for key in pt2_postdict.keys() if key not in badkeys]

In [ ]:
unnormed_likes_pt2 = []
for key in goodkeys_pt2:
    unnormed_likes_pt2.append(pt2_postdict[key]['likelihood_i'])

In [ ]:
plot_aggregate_posterior(Rs,unnormed_likes_pt2,uniform_Rprior,Rtrue=11.98)

In [ ]:
Aplus_4yr_likes = [*unnormed_likes_pt1,*unnormed_likes_pt2]

In [ ]:
plt.figure()
ax = plt.gca()
# plot_aggregate_posterior_on_ax(Rs,likes_sly4,uniform_Rprior,Rtrue=11.54,ax=ax1,legend_loc='upper left',
#                          title='O4+O5 Posterior for sly4 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
plot_aggregate_posterior_on_ax(Rs,Aplus_4yr_likes,uniform_Rprior,Rtrue=11.98,ax=ax,legend_loc='upper left',
                         title='A+ 4yr Posterior for SFHX EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# plot_aggregate_posterior_on_ax(Rs,likes_dd2,uniform_Rprior,Rtrue=13.26,ax=ax3,legend_loc='upper left',
#                          title='O4+O5 Posterior for dd2 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# ax1.set_ylim()
# plt.tight_layout()
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.png',bbox_inches='tight')
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.pdf',bbox_inches='tight')
plt.show()

In [ ]:
prior_stats = get_post_stats(Rprior_kernel_mm.pdf(Rs)/np.sum(Rprior_kernel_mm.pdf(Rs)),Rs,latex=True)
sfhx_stats = get_post_stats(get_posterior(Rs,get_aggregate_likelihood(Aplus_4yr_likes),Rprior_kernel_mm),Rs,latex=True)

In [ ]:
prior_range = prior_stats[2] - prior_stats[1]
sfhx_range = sfhx_stats[2] - sfhx_stats[1]

In [ ]:
prior_range - sfhx_range

In [ ]:
(prior_range - sfhx_range)/prior_range

In [ ]:
get_post_stats(uniform_Rprior.pdf(Rs)/np.sum(uniform_Rprior.pdf(Rs)),Rs,latex=True)

In [ ]:
get_post_stats(get_posterior(Rs,get_aggregate_likelihood(Aplus_4yr_likes),uniform_Rprior),Rs,latex=True)

Just a check: look at what happens when we remove the most nearby events.

In [ ]:
cut = 100 #Mpc
farkeys_pt1 = [key for key in goodkeys_pt1 if float(key.split('_')[1].split('-')[0])>cut]
farkeys_pt2 = [key for key in goodkeys_pt2 if float(key.split('_')[1].split('-')[0])>cut]
farlikes_pt1 = [pt1_postdict[key]['likelihood_i'] for key in farkeys_pt1]
farlikes_pt2 = [pt2_postdict[key]['likelihood_i'] for key in farkeys_pt2]

In [ ]:
pt1_postdict[goodkeys_pt1[2]].keys()
for key1,key2 in zip(goodkeys_pt1,goodkeys_pt2):
    if pt1_postdict[key1]['dist'] < 100:
        print(key1)
    if pt2_postdict[key2]['dist'] < 100:
        print(key2)

Look at the closest (presumeably best) individual posterior:

In [ ]:
plt.figure()
ax = plt.gca()
# plot_aggregate_posterior_on_ax(Rs,likes_sly4,uniform_Rprior,Rtrue=11.54,ax=ax1,legend_loc='upper left',
#                          title='O4+O5 Posterior for sly4 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
plot_aggregate_posterior_on_ax(Rs,[pt2_postdict['129135_53.16']['likelihood_i']],uniform_Rprior,Rtrue=11.98,ax=ax,legend_loc='upper left',
                         title='A+ 4yr Posterior for SFHX EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# plot_aggregate_posterior_on_ax(Rs,likes_dd2,uniform_Rprior,Rtrue=13.26,ax=ax3,legend_loc='upper left',
#                          title='O4+O5 Posterior for dd2 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# ax1.set_ylim()
# plt.tight_layout()
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.png',bbox_inches='tight')
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.pdf',bbox_inches='tight')
plt.show()

In [ ]:
Aplus_4yr_farlikes = [*farlikes_pt1,*farlikes_pt2]

In [ ]:
len(Aplus_4yr_likes) - len(Aplus_4yr_farlikes)

In [ ]:
plt.figure()
ax = plt.gca()
# plot_aggregate_posterior_on_ax(Rs,likes_sly4,uniform_Rprior,Rtrue=11.54,ax=ax1,legend_loc='upper left',
#                          title='O4+O5 Posterior for sly4 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
plot_aggregate_posterior_on_ax(Rs,Aplus_4yr_farlikes,uniform_Rprior,Rtrue=11.98,ax=ax,legend_loc='upper left',
                         title='A+ 4yr Posterior for SFHX EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# plot_aggregate_posterior_on_ax(Rs,likes_dd2,uniform_Rprior,Rtrue=13.26,ax=ax3,legend_loc='upper left',
#                          title='O4+O5 Posterior for dd2 EoS')#,Rticks=[9.5,10,10.5,11,11.5,12,12.5,13,13.5])
# ax1.set_ylim()
# plt.tight_layout()
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.png',bbox_inches='tight')
# plt.savefig('../paper/figures/O4O5_allEoS_3panel_witherr.pdf',bbox_inches='tight')
plt.show()